# Кросс-валидация для временных рядов (пакет [`sktime`](https://www.sktime.net/en/stable/))

Рассмотрим на примере модели ARIMA

In [ ]:
import numpy as np
import pandas as pd

# Модель для прогнозирования, например ARIMA
from sktime.forecasting.arima import ARIMA
# Визуализация временных рядов
from sktime.utils.plotting import plot_series
# Модули для кросс-валидации
from sktime.split import temporal_train_test_split, ExpandingWindowSplitter, SlidingWindowSplitter, SingleWindowSplitter
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError # Метрики MSE, MAE, MAPE

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) месячные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`GS10`](https://fred.stlouisfed.org/series/GS10)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `y`

In [ ]:
y = web.DataReader(name='GS10', data_source='fred', start='2000-01', end='2023-12')
y.index = y.index.to_period(freq='M')
# длина ряда
len(y)

## Валидация через разделение на обучающую и тестовую выборки

Используем метод [`temporal_train_test_split`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.split.temporal_train_test_split.html) из пакета `sktime`

Разделим ряда на обучающую выбору (первые, например 80%, наблюдений) и тестовую выборку (последние, нашем случае 20%, наблюдений)

В качестве метрики выберем [`MeanSquaredError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanSquaredError.html)

Разберём на примере ARIMA(2,1,2)

In [ ]:
# разбиваем выбору на обучающую и тестовую (последние 10 наблюдений)
y_train, y_test = temporal_train_test_split(y, test_size=10)

# инициализируем метрику
metric = MeanSquaredError(square_root=False)

In [ ]:
# специфицируем модель для прогнозирования, например ARIMA(2,1,2)
forecaster = ARIMA(order=(2,1,2), trend='n')

# Обучаем модель на обучающем множестве прогнозируем для тестовых наблюдений
y_pred = forecaster.fit_predict(y_train, fh = y_test.index)

# вычисляем метрику на данных
metric.evaluate(y_test, y_pred)

In [ ]:
plot_series(y_train.tail(30), y_test, y_pred, labels=['train', 'test', 'pred'])

plt.show()

Используем  [`SingleWindowSplitter`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.split.SingleWindowSplitter.html) 

Для вычисления метрики для каждого фолда используем метод [`MeanSquaredError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanSquaredError.html), [`MeanAbsoluteError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanAbsoluteError.html), [`MeanAbsolutePercentageError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanAbsolutePercentageError.html#sktime.performance_metrics.forecasting.MeanAbsolutePercentageError)

Для проведения кросс-валидации используем [`evaluate`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_evaluation.evaluate.html)

Разберём на примере ARMA(2,1,2) без сноса

In [ ]:
# специфицируем модель для прогнозирования, например ARIMA(2,1,2) без сноса
forecaster = ARIMA(order=(2,1,2), trend='n')

# разбиваем выбору на обучающую (параметр window_length) и тестовую (параметр fh)
cv_strategy = SingleWindowSplitter(fh=np.arange(1, 6), window_length=len(y)-5)

# инициализируем метрики
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError(), MeanAbsolutePercentageError()]

cv_res = evaluate(forecaster=forecaster, y=y, cv=cv_strategy, strategy="refit", return_data=False, scoring=metric)
cv_res

In [ ]:
# MSE, MAE, MAPE
cv_res.iloc[:,:len(metric)]

## Валидация методом k-Fold (расширяем обучающую выборку)

Используем  [`ExpandingWindowSplitter`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.split.ExpandingWindowSplitter.html) 

Для вычисления метрики для каждого фолда используем метод [`MeanSquaredError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanSquaredError.html), [`MeanAbsoluteError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanAbsoluteError.html), [`MeanAbsolutePercentageError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanAbsolutePercentageError.html#sktime.performance_metrics.forecasting.MeanAbsolutePercentageError)

Для проведения кросс-валидации используем [`evaluate`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_evaluation.evaluate.html)

Разберём на примере ARIMA(2,1,2) без сноса

In [ ]:
# специфицируем модель для прогнозирования, например ARIMA(2,1,2) без сноса
forecaster = ARIMA(order=(2,1,2), trend='n')

# разбиваем выбору на обучающую (начинаем с первых initial_window) и тестовую длины (параметра fh)
# далее обучающую расширяем на step_length наблюдений
cv_strategy = ExpandingWindowSplitter(fh=np.arange(1, 11), initial_window=100, step_length=10)

# инициализируем метрики
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError(), MeanAbsolutePercentageError()]

cv_res = evaluate(forecaster=forecaster, y=y, cv=cv_strategy, strategy="refit", return_data=False, scoring=metric)
cv_res

In [ ]:
# средняя MSE, MAE, MAPE
cv_res.iloc[:,:len(metric)].mean()

## Валидация методом k-Fold (скользящая обучающая выборка)

Используем метод [`SlidingWindowSplitter`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.split.SlidingWindowSplitter.html)

Для вычисления метрики для каждого фолда используем метод [`MeanSquaredError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanSquaredError.html), [`MeanAbsoluteError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanAbsoluteError.html), [`MeanAbsolutePercentageError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanAbsolutePercentageError.html#sktime.performance_metrics.forecasting.MeanAbsolutePercentageError)

Для проведения кросс-валидации используем [`evaluate`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_evaluation.evaluate.html)

Разберём на примере ARIMA(2,1,2) без сноса

In [ ]:
# специфицируем модель для прогнозирования, например ARIMA(2,1,2) без сноса
forecaster = ARIMA(order=(2,1,2), trend='n')

# разбиваем выбору на обучающую (длины initial_window) и тестовую (параметр fh)
# далее обучающую и тестовую сдвигаем на step_length шагов
cv_strategy = SlidingWindowSplitter(fh=np.arange(1, 11), initial_window=100, step_length=10)

# инициализируем метрики
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError(), MeanAbsolutePercentageError()]

cv_res = evaluate(forecaster=forecaster, y=y, cv=cv_strategy, strategy="refit", return_data=False, scoring=metric)
cv_res

In [ ]:
# средняя MSE, MAE, MAPE
cv_res.iloc[:,:len(metric)].mean()

## Кросс валидация для нескольких моделей

Проведём кросс-валидацию для сравнения нескольких моделей. Например, сравним
* ARIMA(1,1,1) без сноса
* ARIMA(1,1,1) со сносом
* ARIMA(1,2,1) без сноса

In [ ]:
# Зададим список из специфицированных моделей прогнозирования
forecasters = [ARIMA(order=(1,1,1), trend='n'), ARIMA(order=(1,1,1), trend='c'), ARIMA(order=(1,2,1), trend='n')]

# специфицируем метод кросс-валидации. Например, SlidingWindowSplitter
cv_strategy = SlidingWindowSplitter(fh=np.arange(1, 6), initial_window=100, step_length=5)

# инициализируем метрики
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError(), MeanAbsolutePercentageError()]

# датафрейм с метриками по столбцам
cv_data = pd.DataFrame(data=None, columns=['MSE', 'MAE', 'MAPE'])

for model in forecasters:
	print(model)
	cv_res = evaluate(forecaster=model, y=y, cv=cv_strategy, strategy="refit", return_data=False, scoring=metric)
	# print(df.iloc[:,:len(metric)].mean()) # метрики для каждой модели
	cv_data.loc[len(cv_data.index)] = cv_res.iloc[:,[0,1,2]].mean().values

# результаты кросс-валидации в виде датафрейма
cv_data

In [ ]:
# Индекс модели
for i in range(cv_data.shape[1]):
	print(f'{cv_data.columns[i]}: model #={cv_data.iloc[:,i].argmin()}')